In [1]:
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import os
import time
import boto3
import botocore
from pathlib import Path
from huggingface_hub import login
from huggingface_hub import notebook_login

In [2]:
#Some models require a hugginface login; uncomment the following code so you can provide your token

#notebook_login()

#Some models also need you to login to Huggingface using your browser and agree to a use policy.

In [3]:
#Setup variables for this script

In [4]:
models_dir = Path.cwd() / "models"
models_dir.mkdir(parents=True, exist_ok=True)
model_prefix = "GritLM"
model_name = "GritLM-7B"
full_model_name = f"{model_prefix}/{model_name}"
save_path = models_dir / model_prefix

In [5]:
#Download model

In [6]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id=full_model_name, local_dir=save_path)

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/946 [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

'/opt/app-root/src/elastic-3am-killer/model_prep/models/GritLM'

In [7]:
#Get the s3 connection parameters (these are entered in the workbench setup screens)

In [8]:
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

In [9]:
session = boto3.session.Session(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key
          )

In [10]:
s3_resource = session.resource(
               's3',
               config=botocore.client.Config(signature_version='s3v4'),
               endpoint_url=endpoint_url,
               region_name=region_name
              )
bucket = s3_resource.Bucket(bucket_name)
                        
def upload_directory_to_s3(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)
                                
def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [11]:
#Upload the model to an S3 bucket within the cluster

In [12]:
upload_directory_to_s3(save_path, model_name)

/opt/app-root/src/elastic-3am-killer/model_prep/models/GritLM/README.md -> GritLM-7B/README.md
/opt/app-root/src/elastic-3am-killer/model_prep/models/GritLM/generation_config.json -> GritLM-7B/generation_config.json
/opt/app-root/src/elastic-3am-killer/model_prep/models/GritLM/tokenizer_config.json -> GritLM-7B/tokenizer_config.json
/opt/app-root/src/elastic-3am-killer/model_prep/models/GritLM/model-00001-of-00003.safetensors -> GritLM-7B/model-00001-of-00003.safetensors
/opt/app-root/src/elastic-3am-killer/model_prep/models/GritLM/config.json -> GritLM-7B/config.json
/opt/app-root/src/elastic-3am-killer/model_prep/models/GritLM/model.safetensors.index.json -> GritLM-7B/model.safetensors.index.json
/opt/app-root/src/elastic-3am-killer/model_prep/models/GritLM/training_args.bin -> GritLM-7B/training_args.bin
/opt/app-root/src/elastic-3am-killer/model_prep/models/GritLM/model-00003-of-00003.safetensors -> GritLM-7B/model-00003-of-00003.safetensors
/opt/app-root/src/elastic-3am-killer/mod